# Exploring Debugging Symbols

CFiddle can extract [DWARF](https://dwarfstd.org/doc/DWARF4.pdf) debugging information for compiled code.

This support is very rudimentary at the moment.  It's mostly suited for learning how DWARF works.

In [ ]:
%xmode Minimal
from cfiddle import *

In [ ]:
source = code(r"""
extern "C" 
int foo(int a) {
    int j= 1; 
    int k= 2; 
    for(register int i = 0; i < a; i++) {
        k =+i;
    }
    return j + k;
}""")
b = build(source)

## Dump Debugging Information for a Compiled File

This can be very, very long.

In [ ]:
print(b[0].debug_info())

## Or a Just a Function

In [ ]:
print(b[0].debug_info("foo"))

## Extracting Stack Frames

CFiddle has basic support for extracting the call frame for a function.  Which makes some sense if you look at it next to the source and the assembly.

The DWARF data tell us, for instance, that `a` is at offset -44 relative to the DWARF's concept of the base of the stack frame.  Simultaneously, the assembly suggests that `a` is  at offset -28 relative to `%rbp`, which is the architecture's notion of the base of the stack frame -- a difference of 16.  If you add 16 to the other offsets in stack frame output, the results line up with the assembly:  `j` is at -12, and `k` is at -16.  `i` is in a register.

Remarkably, DWARF4 includes a Turing-complete language for computing these and other values.  See, CFiddle taught you something already!

Obviously, there's more work to be done in in CFiddle to make the stack frames easier to understand.

In [ ]:
from cfiddle.jupyter import compare
compare([b[0].stack_frame("foo"), b[0].source(), b[0].asm("foo")])